# 导包

In [291]:
import datetime 
import pandas as pd
import numpy as np
from math import log,sqrt,exp
from scipy import stats
import plotly.graph_objects as go
import plotly
import plotly.express as px
import re
plotly.offline.init_notebook_mode(connected=True)

# 50ETF期权日行情2017-2018日行情数据

In [299]:
df = pd.read_csv('./data/50ETF期权日行情2017-2018.csv',usecols=['date','trade_code','涨跌幅(%)','收盘价','成交量'])
df2016 = pd.read_csv('./data/50ETF期权日行情2015-2016.csv',usecols=['date','trade_code','涨跌幅(%)','收盘价','成交量'],encoding='gbk')
df = df.append(df2016)

In [300]:
df = df.sort_values('date').reset_index(drop=True)

In [301]:
df

,date,trade_code,涨跌幅(%),收盘价,成交量
0,2015/10/12,510050P1510M01900,-32.56,0.0029,"2,665"
1,2015/10/12,510050C1603M02050,25.94,0.3277,6
2,2015/10/12,510050P1603M01950,-14.71,0.0713,159
3,2015/10/12,510050P1603M02000,-21.06,0.0817,202
4,2015/10/12,510050P1603M02050,-20.78,0.0980,80
...,...,...,...,...,...
98749,2018/9/7,510050P1809M02450,-23.97,0.0368,"130,891"
98750,2018/9/7,510050C1809M02450,17.49,0.0813,"84,512"
98751,2018/9/7,510050P1809M02500,-21.38,0.0581,"143,618"
98752,2018/9/7,510050P1812M02550,-8.72,0.1560,"1,040"


In [302]:
df['K'] = [int(i[-4:])/1000 for i in df.trade_code] # 获取行权价格
df['call_put'] = [i[6] for i in df.trade_code]  # 分离看涨、看跌
df['Adjust'] = [i[11] for i in df.trade_code]  # 分离看涨、看跌
df.date = pd.to_datetime(df.date) # 转变日期格式
# pd.to_numeric(df['成交量'])
df['成交量'] = df['成交量'].astype('str').apply(lambda x: x.replace(',','')).astype('int')

# 成交量剔除

In [303]:
df[df['成交量'] < df['成交量'].quantile(0.1)]

,date,trade_code,涨跌幅(%),收盘价,成交量,K,call_put,Adjust
1,2015-10-12,510050C1603M02050,25.94,0.3277,6,2.05,C,M
7,2015-10-12,510050C1603M02000,23.59,0.3562,38,2.00,C,M
13,2015-10-12,510050C1603M01850,14.63,0.4653,35,1.85,C,M
14,2015-10-12,510050C1603M01900,18.47,0.4144,10,1.90,C,M
17,2015-10-12,510050C1603M01950,23.87,0.3892,10,1.95,C,M
...,...,...,...,...,...,...,...,...
97531,2018-09-17,510050P1903M02750,8.70,0.3237,21,2.75,P,M
97557,2018-09-17,510050C1903M02300,-6.47,0.2630,28,2.30,C,M
97561,2018-09-17,510050P1903M02700,4.12,0.2856,35,2.70,P,M
97897,2018-09-20,510050C1903M02400,-0.68,0.2480,31,2.40,C,M


In [304]:
df = df[df['成交量'] > df['成交量'].quantile(0.1)]

In [305]:
df

,date,trade_code,涨跌幅(%),收盘价,成交量,K,call_put,Adjust
0,2015-10-12,510050P1510M01900,-32.56,0.0029,2665,1.90,P,M
2,2015-10-12,510050P1603M01950,-14.71,0.0713,159,1.95,P,M
3,2015-10-12,510050P1603M02000,-21.06,0.0817,202,2.00,P,M
4,2015-10-12,510050P1603M02050,-20.78,0.0980,80,2.05,P,M
5,2015-10-12,510050P1603M02100,-18.60,0.1199,134,2.10,P,M
...,...,...,...,...,...,...,...,...
98749,2018-09-07,510050P1809M02450,-23.97,0.0368,130891,2.45,P,M
98750,2018-09-07,510050C1809M02450,17.49,0.0813,84512,2.45,C,M
98751,2018-09-07,510050P1809M02500,-21.38,0.0581,143618,2.50,P,M
98752,2018-09-07,510050P1812M02550,-8.72,0.1560,1040,2.55,P,M


# 收盘价剔除

In [306]:
df[df['收盘价'] < 0.0001]

,date,trade_code,涨跌幅(%),收盘价,成交量,K,call_put,Adjust


In [307]:
df = df[df['收盘价'] > 0.001]
df

,date,trade_code,涨跌幅(%),收盘价,成交量,K,call_put,Adjust
0,2015-10-12,510050P1510M01900,-32.56,0.0029,2665,1.90,P,M
2,2015-10-12,510050P1603M01950,-14.71,0.0713,159,1.95,P,M
3,2015-10-12,510050P1603M02000,-21.06,0.0817,202,2.00,P,M
4,2015-10-12,510050P1603M02050,-20.78,0.0980,80,2.05,P,M
5,2015-10-12,510050P1603M02100,-18.60,0.1199,134,2.10,P,M
...,...,...,...,...,...,...,...,...
98749,2018-09-07,510050P1809M02450,-23.97,0.0368,130891,2.45,P,M
98750,2018-09-07,510050C1809M02450,17.49,0.0813,84512,2.45,C,M
98751,2018-09-07,510050P1809M02500,-21.38,0.0581,143618,2.50,P,M
98752,2018-09-07,510050P1812M02550,-8.72,0.1560,1040,2.55,P,M


# 标的资产价格 50ETF

In [308]:
df_d = pd.read_csv('./data/50ETF基金净值表现日数据.csv',usecols=['date','close'],encoding='gbk')

In [309]:
df_d[df_d[df_d.date=='2015-02-09'].index.tolist()[0]: (df_d[df_d.date=='2018-09-21'].index.tolist()[0]+1)]
df_d.date = pd.to_datetime(df_d.date) # 转变日期格式

In [310]:
df = pd.merge(df,df_d,on='date',how='left')

In [311]:
df

,date,trade_code,涨跌幅(%),收盘价,成交量,K,call_put,Adjust,close
0,2015-10-12,510050P1510M01900,-32.56,0.0029,2665,1.90,P,M,2.288
1,2015-10-12,510050P1603M01950,-14.71,0.0713,159,1.95,P,M,2.288
2,2015-10-12,510050P1603M02000,-21.06,0.0817,202,2.00,P,M,2.288
3,2015-10-12,510050P1603M02050,-20.78,0.0980,80,2.05,P,M,2.288
4,2015-10-12,510050P1603M02100,-18.60,0.1199,134,2.10,P,M,2.288
...,...,...,...,...,...,...,...,...,...
83294,2018-09-07,510050P1809M02450,-23.97,0.0368,130891,2.45,P,M,2.491
83295,2018-09-07,510050C1809M02450,17.49,0.0813,84512,2.45,C,M,2.491
83296,2018-09-07,510050P1809M02500,-21.38,0.0581,143618,2.50,P,M,2.491
83297,2018-09-07,510050P1812M02550,-8.72,0.1560,1040,2.55,P,M,2.491


# 无风险利率替代品 shibor一周利率

In [312]:
df2 = pd.read_csv('./data/shibor.csv',usecols=['date','1w']) # 2017年
df3 = pd.read_csv('./data/shibor2.csv',usecols=['date','1w']) # 2018年
df4 = pd.read_csv('./data/shibor3.csv',usecols=['date','1w']) # 2015年
df5 = pd.read_csv('./data/shibor4.csv',usecols=['date','1w']) # 2016年
df_shibor = df2.append([df3,df4,df5]) # 拼接两年
df_shibor = df_shibor.sort_values(by='date',ascending=True).reset_index(drop=True) # 重新排序
df_shibor.date = df_shibor.date.apply(lambda x: str(x)[:4]+'/'+str(x)[4:6]+'/'+str(x)[6:])# 日期格式
df_shibor.date = pd.to_datetime(df_shibor.date) # 转换时间格式

In [313]:
df['shibor'] = df.date.apply(lambda x: (df_shibor[df_shibor.date==x]['1w'].values[0])/100) # 进行配对，将shibor利率放入df

#df['shibor'] = df['shibor'].apply(lambda x: x/100)
#df = df.sort_values(by='date').reset_index(drop=True) # 升序排列

In [314]:
df

,date,trade_code,涨跌幅(%),收盘价,成交量,K,call_put,Adjust,close,shibor
0,2015-10-12,510050P1510M01900,-32.56,0.0029,2665,1.90,P,M,2.288,0.02397
1,2015-10-12,510050P1603M01950,-14.71,0.0713,159,1.95,P,M,2.288,0.02397
2,2015-10-12,510050P1603M02000,-21.06,0.0817,202,2.00,P,M,2.288,0.02397
3,2015-10-12,510050P1603M02050,-20.78,0.0980,80,2.05,P,M,2.288,0.02397
4,2015-10-12,510050P1603M02100,-18.60,0.1199,134,2.10,P,M,2.288,0.02397
...,...,...,...,...,...,...,...,...,...,...
83294,2018-09-07,510050P1809M02450,-23.97,0.0368,130891,2.45,P,M,2.491,0.02673
83295,2018-09-07,510050C1809M02450,17.49,0.0813,84512,2.45,C,M,2.491,0.02673
83296,2018-09-07,510050P1809M02500,-21.38,0.0581,143618,2.50,P,M,2.491,0.02673
83297,2018-09-07,510050P1812M02550,-8.72,0.1560,1040,2.55,P,M,2.491,0.02673


## 到期日

In [315]:
df['t'] = [i[-10:-6] for i in df.trade_code]
fd = df.groupby('t').count().index.to_list()

In [316]:
def prompt_day(fd):
    td = []
    for d in fd:
        y = int('20' + d[:2]) 
        m = int(d[2:])
        if m != 12:
            ndays = (datetime.date(y, m + 1, 1) - datetime.date(y, m, 1)).days
        else: ndays = 31
        date_list = []
        for i in range(1,ndays+1):
            p = datetime.date(y,m,i)
            nweek = p.isoweekday() #返回数字1-7，1为周一,7为周日
            if nweek == 3:    # 交割日为每月第四周星期三
                d2 = p.strftime('%Y%m%d')
                date_list.append(d2)
            if i == ndays:
                td.append(date_list[3])
                date_list=[]  # 清空
    return td
td = prompt_day(fd)

In [329]:
df2 = df.copy()
df2.t = pd.to_datetime([i for x in df2.t for i in td if re.search(x,i) != None])
df2.t = df2.t - df2.date
df2[df2.t.astype('str').apply(lambda x:x[:-5]).astype('int32') < 7]

,date,trade_code,涨跌幅(%),收盘价,成交量,K,call_put,Adjust,close,shibor,t
813,2015-10-22,510050C1510M01850,0.87,0.4755,179,1.85,C,M,2.333,0.02396,6 days
814,2015-10-22,510050C1510M01900,1.09,0.4262,181,1.90,C,M,2.333,0.02396,6 days
815,2015-10-22,510050C1510M01950,1.00,0.3752,137,1.95,C,M,2.333,0.02396,6 days
816,2015-10-22,510050C1510M02000,0.28,0.3227,454,2.00,C,M,2.333,0.02396,6 days
817,2015-10-22,510050C1510M02050,0.96,0.2747,238,2.05,C,M,2.333,0.02396,6 days
...,...,...,...,...,...,...,...,...,...,...,...
82701,2018-09-21,510050P1809M02700,-53.14,0.0745,13948,2.70,P,M,2.632,0.02654,5 days
82702,2018-09-21,510050C1809M02550,303.61,0.0783,243099,2.55,C,M,2.632,0.02654,5 days
82703,2018-09-21,510050C1809M02500,148.02,0.1250,156750,2.50,C,M,2.632,0.02654,5 days
82704,2018-09-21,510050P1809M02550,-91.49,0.0024,169919,2.55,P,M,2.632,0.02654,5 days


# 剔除交割日期少于7天

In [330]:
df2.loc[:,'t'] = [i[-10:-6] for i in df2.trade_code]
import re
df2.t = pd.to_datetime([i for x in df2.t for i in td if re.search(x,i) != None])
df2.t = df2.t - df2.date
print('到期日小于7天的合约数有{}个。'.format(df2[df2.t.astype('str').apply(lambda x:x[:-5]).astype('int32') < 7].t.shape[0]))
df2 = df2[df2.t.astype('str').apply(lambda x:x[:-5]).astype('int32') > 7]
df2.loc[:,'t'] = df2.t.astype('str').apply(lambda x:x[:-5]).astype('float64') / 365
#df['t'] = df2.t.astype('str').apply(lambda x:x[:-5]).astype('float64') / 365

到期日小于7天的合约数有3885个。


In [331]:
df2

,date,trade_code,涨跌幅(%),收盘价,成交量,K,call_put,Adjust,close,shibor,t
0,2015-10-12,510050P1510M01900,-32.56,0.0029,2665,1.90,P,M,2.288,0.02397,0.043836
1,2015-10-12,510050P1603M01950,-14.71,0.0713,159,1.95,P,M,2.288,0.02397,0.446575
2,2015-10-12,510050P1603M02000,-21.06,0.0817,202,2.00,P,M,2.288,0.02397,0.446575
3,2015-10-12,510050P1603M02050,-20.78,0.0980,80,2.05,P,M,2.288,0.02397,0.446575
4,2015-10-12,510050P1603M02100,-18.60,0.1199,134,2.10,P,M,2.288,0.02397,0.446575
...,...,...,...,...,...,...,...,...,...,...,...
83294,2018-09-07,510050P1809M02450,-23.97,0.0368,130891,2.45,P,M,2.491,0.02673,0.052055
83295,2018-09-07,510050C1809M02450,17.49,0.0813,84512,2.45,C,M,2.491,0.02673,0.052055
83296,2018-09-07,510050P1809M02500,-21.38,0.0581,143618,2.50,P,M,2.491,0.02673,0.052055
83297,2018-09-07,510050P1812M02550,-8.72,0.1560,1040,2.55,P,M,2.491,0.02673,0.301370


# 条件约束

In [332]:
pp = df2.copy()

In [333]:
def cons(df): # 计算隐含分红率
        #q = -log((df.settle+df.exercise_price*exp(-df.interest*df.delta)-df.settle_p)/(df.s0))/df.delta
    if df['call_put'] == 'C':
        price = df.close - df.K*exp(-df.shibor*df.t)
        price = max(price,0)
        if price < df['收盘价'] and df['收盘价'] < df.close:
            return 1
        else: return 0 
    else:
        price =  df.K*exp(-df.shibor/100*df.t) - df.close
        price = max(price,0)
        if price < df['收盘价'] and df['收盘价'] < df.K*exp(-df.shibor/100*df.t):
            return 1
        else: return 0 

In [334]:
pp['cons']=pp.apply(cons,axis=1)
pp[pp['cons'] == 0]

,date,trade_code,涨跌幅(%),收盘价,成交量,K,call_put,Adjust,close,shibor,t,cons
5,2015-10-12,510050C1512M01800,2.86,0.4633,1864,1.80,C,M,2.288,0.02397,0.197260,0
6,2015-10-12,510050C1512M01850,5.59,0.4286,642,1.85,C,M,2.288,0.02397,0.197260,0
10,2015-10-12,510050C1603M01800,3.68,0.5038,118,1.80,C,M,2.288,0.02397,0.446575,0
11,2015-10-12,510050C1512M01900,16.57,0.3828,58,1.90,C,M,2.288,0.02397,0.197260,0
22,2015-10-12,510050C1512M01950,21.44,0.3398,56,1.95,C,M,2.288,0.02397,0.197260,0
...,...,...,...,...,...,...,...,...,...,...,...,...
83262,2018-09-07,510050P1809M03000,-5.34,0.5055,399,3.00,P,M,2.491,0.02673,0.052055,0
83263,2018-09-07,510050P1809M02950,-6.03,0.4548,331,2.95,P,M,2.491,0.02673,0.052055,0
83264,2018-09-07,510050P1809M02900,-6.71,0.4049,553,2.90,P,M,2.491,0.02673,0.052055,0
83265,2018-09-07,510050P1809M03200,-3.73,0.7066,454,3.20,P,M,2.491,0.02673,0.052055,0


In [335]:
df2 = pp[pp['cons'] == 1]
df2

,date,trade_code,涨跌幅(%),收盘价,成交量,K,call_put,Adjust,close,shibor,t,cons
0,2015-10-12,510050P1510M01900,-32.56,0.0029,2665,1.90,P,M,2.288,0.02397,0.043836,1
1,2015-10-12,510050P1603M01950,-14.71,0.0713,159,1.95,P,M,2.288,0.02397,0.446575,1
2,2015-10-12,510050P1603M02000,-21.06,0.0817,202,2.00,P,M,2.288,0.02397,0.446575,1
3,2015-10-12,510050P1603M02050,-20.78,0.0980,80,2.05,P,M,2.288,0.02397,0.446575,1
4,2015-10-12,510050P1603M02100,-18.60,0.1199,134,2.10,P,M,2.288,0.02397,0.446575,1
...,...,...,...,...,...,...,...,...,...,...,...,...
83293,2018-09-07,510050C1812M02500,9.59,0.1325,1445,2.50,C,M,2.491,0.02673,0.301370,1
83294,2018-09-07,510050P1809M02450,-23.97,0.0368,130891,2.45,P,M,2.491,0.02673,0.052055,1
83295,2018-09-07,510050C1809M02450,17.49,0.0813,84512,2.45,C,M,2.491,0.02673,0.052055,1
83296,2018-09-07,510050P1809M02500,-21.38,0.0581,143618,2.50,P,M,2.491,0.02673,0.052055,1


# 数据清洗

In [213]:
df = df2[['date','trade_code','call_put','t','K','收盘价','close','shibor']]
df = df.rename(columns={'date':'trade_date','收盘价':'c','trade_code':'ts_code','shibor':'r','close':'s','K':'k'})
from math import log,sqrt,exp
# https://zhuanlan.zhihu.com/p/142685333
def data_clear(df): # 数据清洗
    def iq(df): # 计算隐含分红率
        #q = -log((df.settle+df.exercise_price*exp(-df.interest*df.delta)-df.settle_p)/(df.s0))/df.delta
        q = -log((df.c+df.k*exp(-df.r*df.t)-df.c_p)/(df.s))/df.t
        return q

    # 计算隐含分红率
    df_c = df[df['call_put']=='C']
    # print(df_c.shape)
    df_p = df[df['call_put']=='P']
    def P_C(df):
        df.ts_code = df.ts_code[:6] + 'P' + df.ts_code[-10:]
        return df
    df_c = df_c.apply(P_C,axis = 1)
    # print(df_p.shape)
    df_p = df_p.rename(columns={'c':'c_p',
                         'call_put':'call_put_p'})
    df = pd.merge(df_c,df_p,how='left',on=['trade_date','ts_code','k','t','r','s'])
    # print(df.shape)
    df['q'] = df.apply(iq,axis = 1)
    c_list = [x for x in range(8)]+[10]
    
    df_c = df.iloc[:,c_list]
    df_p = df[['trade_date','ts_code','c_p','call_put_p','k','t','r','s','q']]
    df_p = df_p.rename(columns={'c_p':'c',
                         'call_put_p':'call_put'})
    df_c = df_c.append(df_p)
    #print(df_c)
    return df_c        

In [199]:
df = df2[['date','trade_code','call_put','t','K','收盘价','close','shibor']]
df = df.rename(columns={'date':'trade_date','收盘价':'c','trade_code':'ts_code','shibor':'r','close':'s','K':'k'})
def data_clear(df): # 数据清洗
    def iq(df): # 计算隐含分红率
        #q = -log((df.settle+df.exercise_price*exp(-df.interest*df.delta)-df.settle_p)/(df.s0))/df.delta
        q = -log((df.c+df.k*exp(-df.r*df.t)-df.c_p)/(df.s))/df.t
        return q

    # 计算隐含分红率
    df_c = df[df['call_put']=='C']
    df_p = df[df['call_put']=='P']
    df_p = df_p.rename(columns={'c':'c_p','ts_code':'ts_code_p',
                         'call_put':'call_put_p'})
    df = pd.merge(df_c,df_p,how='left',on=['trade_date','k','t','r','s'])

    df['q'] = df.apply(iq,axis = 1)
    c_list = [x for x in range(8)]+[11]
    
    df_c = df.iloc[:,c_list]
    df_p = df[['ts_code_p','trade_date','c_p','call_put_p','k','t','r','s','q']]
    df_p = df_p.rename(columns={'c_p':'c','ts_code_p':'ts_code',
                         'call_put_p':'call_put'})
    df_c = df_c.append(df_p)
    #print(df_c)
    return df_c        

In [194]:
df_c = data_clear(df)

In [214]:
df_c = df[df['call_put']=='C']
df_p = df[df['call_put']=='P']
df = pd.merge(df_c,df_p,how='left',on=['trade_date','k','t','r','s'])

In [217]:
df_p[(df_p['ts_code']=='510050P1612A02450') & (df_p['trade_date']=='2016-11-29')]

,trade_date,ts_code,call_put,t,k,c,s,r
36461,2016-11-29,510050P1612A02450,P,0.079452,2.45,0.0248,2.446,0.02481


In [218]:
df_p[(df_p['ts_code']=='510050P1612M02450') & (df_p['trade_date']=='2016-11-29')]

,trade_date,ts_code,call_put,t,k,c,s,r
36366,2016-11-29,510050P1612M02450,P,0.079452,2.45,0.0456,2.446,0.02481


In [215]:
df[df.trade_date == '2016-11-29']

,trade_date,ts_code_x,call_put_x,t,k,c_x,s,r,ts_code_y,call_put_y,c_y
14859,2016-11-29,510050C1612M02450,C,0.079452,2.45,0.0369,2.446,0.02481,510050P1612M02450,P,0.0456
14860,2016-11-29,510050C1612M02450,C,0.079452,2.45,0.0369,2.446,0.02481,510050P1612A02450,P,0.0248
14861,2016-11-29,510050C1612M02500,C,0.079452,2.50,0.0232,2.446,0.02481,510050P1612M02500,P,0.0811
14862,2016-11-29,510050C1612M02400,C,0.079452,2.40,0.0648,2.446,0.02481,510050P1612M02400,P,0.0248
14863,2016-11-29,510050C1612M02400,C,0.079452,2.40,0.0648,2.446,0.02481,510050P1612A02400,P,0.0141
...,...,...,...,...,...,...,...,...,...,...,...
14943,2016-11-29,510050C1706A02350,C,0.578082,2.35,0.1862,2.446,0.02481,510050P1706M02350,P,0.0920
14944,2016-11-29,510050C1706A02350,C,0.578082,2.35,0.1862,2.446,0.02481,510050P1706A02350,P,0.0696
14945,2016-11-29,510050C1706A02400,C,0.578082,2.40,0.1580,2.446,0.02481,510050P1706M02400,P,0.1147
14946,2016-11-29,510050C1706A02400,C,0.578082,2.40,0.1580,2.446,0.02481,510050P1706A02400,P,0.0897


In [197]:
df_put[(df_put.ts_code == '510050P1612A02450') & (df_put.trade_date == '2016-11-29')]

,trade_date,ts_code,call_put,t,k,c,s,r
36461,2016-11-29,510050P1612A02450,P,0.079452,2.45,0.0248,2.446,0.02481


In [187]:
df_put[df_put.trade_date == '2016-11-29']

,trade_date,ts_code,call_put,t,k,c,s,r
36363,2016-11-29,510050P1612M02300,P,0.079452,2.30,0.0068,2.446,0.02481
36364,2016-11-29,510050P1612M02350,P,0.079452,2.35,0.0140,2.446,0.02481
36365,2016-11-29,510050P1612M02500,P,0.079452,2.50,0.0811,2.446,0.02481
36366,2016-11-29,510050P1612M02450,P,0.079452,2.45,0.0456,2.446,0.02481
36370,2016-11-29,510050P1612M02400,P,0.079452,2.40,0.0248,2.446,0.02481
36372,2016-11-29,510050P1701A02500,P,0.156164,2.50,0.0673,2.446,0.02481
36373,2016-11-29,510050P1706A02550,P,0.578082,2.55,0.1711,2.446,0.02481
36374,2016-11-29,510050P1703A02550,P,0.309589,2.55,0.1333,2.446,0.02481
36380,2016-11-29,510050P1701A02450,P,0.156164,2.45,0.0431,2.446,0.02481
36381,2016-11-29,510050P1701A02400,P,0.156164,2.40,0.0264,2.446,0.02481


In [188]:
df_c[df_c.index==14859]

,trade_date,ts_code,call_put,t,k,c,s,r,q
14859,2016-11-29,510050C1612M02450,C,0.079452,2.45,0.0369,2.446,0.02481,0.049106
14859,2016-11-29,510050P1612M02450,P,0.079452,2.45,0.0456,2.446,0.02481,0.049106


In [189]:
df[(df.ts_code == '510050P1612M02450') & (df.trade_date == '2016-11-29')]

,trade_date,ts_code,call_put,t,k,c,s,r
36366,2016-11-29,510050P1612M02450,P,0.079452,2.45,0.0456,2.446,0.02481


In [190]:
df[(df.ts_code == '510050C1612M02450') & (df.trade_date == '2016-11-29')]

,trade_date,ts_code,call_put,t,k,c,s,r
36361,2016-11-29,510050C1612M02450,C,0.079452,2.45,0.0369,2.446,0.02481


In [169]:
df_c[(df_c.ts_code == '510050P1612M02450') & (df_c.trade_date == '2016-11-29')]

,trade_date,ts_code,call_put,t,k,c,s,r,q
14859,2016-11-29,510050P1612M02450,P,0.079452,2.45,0.0456,2.446,0.02481,0.049106
14925,2016-11-29,510050P1612M02450,P,0.079452,2.45,0.0456,2.446,0.02481,-0.098284


In [171]:
df_c[(df_c.ts_code == '510050C1612M02450') & (df_c.trade_date == '2016-11-29')]

,trade_date,ts_code,call_put,t,k,c,s,r,q
14859,2016-11-29,510050C1612M02450,C,0.079452,2.45,0.0369,2.446,0.02481,0.049106
14860,2016-11-29,510050C1612M02450,C,0.079452,2.45,0.0369,2.446,0.02481,-0.057885


In [172]:
df_c[(df_c.ts_code == '510050C1612A02450') & (df_c.trade_date == '2016-11-29')]

,trade_date,ts_code,call_put,t,k,c,s,r,q
14925,2016-11-29,510050C1612A02450,C,0.079452,2.45,0.0656,2.446,0.02481,-0.098284
14926,2016-11-29,510050C1612A02450,C,0.079452,2.45,0.0656,2.446,0.02481,-0.204035


In [170]:
df_c[(df_c.ts_code == '510050P1612A02450') & (df_c.trade_date == '2016-11-29')]

,trade_date,ts_code,call_put,t,k,c,s,r,q
14860,2016-11-29,510050P1612A02450,P,0.079452,2.45,0.0248,2.446,0.02481,-0.057885
14926,2016-11-29,510050P1612A02450,P,0.079452,2.45,0.0248,2.446,0.02481,-0.204035


In [163]:
df_c[df_c.duplicated(subset=['trade_date','t','k','s','r'])==True]

,trade_date,ts_code,call_put,t,k,c,s,r,q
14860,2016-11-29,510050C1612M02450,C,0.079452,2.45,0.0369,2.446,0.02481,-0.057885
14863,2016-11-29,510050C1612M02400,C,0.079452,2.40,0.0648,2.446,0.02481,0.000135
14865,2016-11-29,510050C1701M02350,C,0.156164,2.35,0.1130,2.446,0.02481,0.050430
14867,2016-11-29,510050C1612M02350,C,0.079452,2.35,0.1027,2.446,0.02481,0.025382
14872,2016-11-29,510050C1701M02400,C,0.156164,2.40,0.0802,2.446,0.02481,0.052259
...,...,...,...,...,...,...,...,...,...
40799,2018-09-21,510050P1812M02450,P,0.263014,2.45,0.0411,2.632,0.02654,0.008883
40800,2018-09-21,510050P1812M02900,P,0.263014,2.90,0.2815,2.632,0.02654,0.008638
40801,2018-09-21,510050P1812M02850,P,0.263014,2.85,0.2411,2.632,0.02654,0.008713
40802,2018-09-21,510050P1812M02750,P,0.263014,2.75,0.1717,2.632,0.02654,0.009588


In [ ]:
df_c[df_c.tr]

In [141]:
df = df2[['date','trade_code','call_put','t','K','收盘价','close','shibor']]
df = df.rename(columns={'date':'trade_date','收盘价':'c','trade_code':'ts_code','shibor':'r','close':'s','K':'k'})
df_c = df[df['call_put']=='C']
df_p = df[df['call_put']=='P']
df_p = df_p.rename(columns={'c':'c_p','ts_code':'ts_code_p',
                         'call_put':'call_put_p'})

In [160]:
df_p[df_p.duplicated(subset=['trade_date','t','k','s','r'])==True]

,trade_date,ts_code_p,call_put_p,t,k,c_p,s,r
36396,2016-11-29,510050P1701A02300,P,0.156164,2.30,0.0082,2.446,0.02481
36408,2016-11-29,510050P1701M02500,P,0.156164,2.50,0.1007,2.446,0.02481
36409,2016-11-29,510050P1701M02450,P,0.156164,2.45,0.0694,2.446,0.02481
36410,2016-11-29,510050P1701M02400,P,0.156164,2.40,0.0448,2.446,0.02481
36411,2016-11-29,510050P1701M02350,P,0.156164,2.35,0.0271,2.446,0.02481
...,...,...,...,...,...,...,...,...
74551,2018-06-11,510050P1806A02700,P,0.043836,2.70,0.0377,2.663,0.02776
74680,2018-06-12,510050P1806A02700,P,0.041096,2.70,0.0261,2.688,0.02789
74807,2018-06-13,510050P1806A02700,P,0.038356,2.70,0.0314,2.672,0.02800
74933,2018-06-14,510050P1806A02700,P,0.035616,2.70,0.0288,2.675,0.02809


In [153]:
df_c

,trade_date,ts_code,call_put,t,k,c,s,r
0,2015-02-09,510050C1503M02200,C,0.120548,2.20,0.1826,2.331,0.04335
1,2015-02-09,510050C1506M02300,C,0.369863,2.30,0.2105,2.331,0.04335
2,2015-02-09,510050C1506M02350,C,0.369863,2.35,0.1870,2.331,0.04335
3,2015-02-09,510050C1506M02400,C,0.369863,2.40,0.1652,2.331,0.04335
9,2015-02-09,510050C1509M02200,C,0.619178,2.20,0.3130,2.331,0.04335
...,...,...,...,...,...,...,...,...
83285,2018-09-21,510050C1812M02450,C,0.263014,2.45,0.2340,2.632,0.02654
83294,2018-09-21,510050C1812M02900,C,0.263014,2.90,0.0277,2.632,0.02654
83295,2018-09-21,510050C1812M02850,C,0.263014,2.85,0.0369,2.632,0.02654
83296,2018-09-21,510050C1812M02750,C,0.263014,2.75,0.0662,2.632,0.02654


In [144]:
df

,trade_date,ts_code,call_put,t,k,c,s,r,ts_code_p,call_put_p,c_p
0,2015-02-09,510050C1503M02200,C,0.120548,2.20,0.1826,2.331,0.04335,510050P1503M02200,P,0.0617
1,2015-02-09,510050C1506M02300,C,0.369863,2.30,0.2105,2.331,0.04335,510050P1506M02300,P,0.1882
2,2015-02-09,510050C1506M02350,C,0.369863,2.35,0.1870,2.331,0.04335,510050P1506M02350,P,0.2055
3,2015-02-09,510050C1506M02400,C,0.369863,2.40,0.1652,2.331,0.04335,510050P1506M02400,P,0.2329
4,2015-02-09,510050C1509M02200,C,0.619178,2.20,0.3130,2.331,0.04335,510050P1509M02200,P,0.1795
...,...,...,...,...,...,...,...,...,...,...,...
40799,2018-09-21,510050C1812M02450,C,0.263014,2.45,0.2340,2.632,0.02654,510050P1812M02450,P,0.0411
40800,2018-09-21,510050C1812M02900,C,0.263014,2.90,0.0277,2.632,0.02654,510050P1812M02900,P,0.2815
40801,2018-09-21,510050C1812M02850,C,0.263014,2.85,0.0369,2.632,0.02654,510050P1812M02850,P,0.2411
40802,2018-09-21,510050C1812M02750,C,0.263014,2.75,0.0662,2.632,0.02654,510050P1812M02750,P,0.1717


In [137]:
df = pd.merge(df_c,df_p,how='left',on=['trade_date','k','t','r','s'])
# df[df.isnull().T.any()]
df[df.isnull().values==False]


,trade_date,ts_code,call_put,t,k,c,s,r,ts_code_p,call_put_p,c_p
0,2015-02-09,510050C1503M02200,C,0.120548,2.2,0.1826,2.331,0.04335,510050P1503M02200,P,0.0617
0,2015-02-09,510050C1503M02200,C,0.120548,2.2,0.1826,2.331,0.04335,510050P1503M02200,P,0.0617
0,2015-02-09,510050C1503M02200,C,0.120548,2.2,0.1826,2.331,0.04335,510050P1503M02200,P,0.0617
0,2015-02-09,510050C1503M02200,C,0.120548,2.2,0.1826,2.331,0.04335,510050P1503M02200,P,0.0617
0,2015-02-09,510050C1503M02200,C,0.120548,2.2,0.1826,2.331,0.04335,510050P1503M02200,P,0.0617
...,...,...,...,...,...,...,...,...,...,...,...
40803,2018-09-21,510050C1812M02800,C,0.263014,2.8,0.0501,2.632,0.02654,510050P1812M02800,P,0.2041
40803,2018-09-21,510050C1812M02800,C,0.263014,2.8,0.0501,2.632,0.02654,510050P1812M02800,P,0.2041
40803,2018-09-21,510050C1812M02800,C,0.263014,2.8,0.0501,2.632,0.02654,510050P1812M02800,P,0.2041
40803,2018-09-21,510050C1812M02800,C,0.263014,2.8,0.0501,2.632,0.02654,510050P1812M02800,P,0.2041


In [138]:
df[df.isnull().values==True]

,trade_date,ts_code,call_put,t,k,c,s,r,ts_code_p,call_put_p,c_p
128,2015-02-16,510050C1506M02500,C,0.350685,2.50,0.0979,2.394,0.04700,NaN,NaN,NaN
128,2015-02-16,510050C1506M02500,C,0.350685,2.50,0.0979,2.394,0.04700,NaN,NaN,NaN
128,2015-02-16,510050C1506M02500,C,0.350685,2.50,0.0979,2.394,0.04700,NaN,NaN,NaN
130,2015-02-16,510050C1506M02450,C,0.350685,2.45,0.1166,2.394,0.04700,NaN,NaN,NaN
130,2015-02-16,510050C1506M02450,C,0.350685,2.45,0.1166,2.394,0.04700,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
40762,2018-09-20,510050C1812M02950,C,0.265753,2.95,0.0110,2.541,0.02659,NaN,NaN,NaN
40762,2018-09-20,510050C1812M02950,C,0.265753,2.95,0.0110,2.541,0.02659,NaN,NaN,NaN
40766,2018-09-20,510050C1812M02900,C,0.265753,2.90,0.0150,2.541,0.02659,NaN,NaN,NaN
40766,2018-09-20,510050C1812M02900,C,0.265753,2.90,0.0150,2.541,0.02659,NaN,NaN,NaN


In [139]:
df[df.isnull().T.any()]

,trade_date,ts_code,call_put,t,k,c,s,r,ts_code_p,call_put_p,c_p
128,2015-02-16,510050C1506M02500,C,0.350685,2.50,0.0979,2.394,0.04700,NaN,NaN,NaN
130,2015-02-16,510050C1506M02450,C,0.350685,2.45,0.1166,2.394,0.04700,NaN,NaN,NaN
132,2015-02-16,510050C1506M02400,C,0.350685,2.40,0.1390,2.394,0.04700,NaN,NaN,NaN
174,2015-02-25,510050C1509M02350,C,0.575342,2.35,0.2001,2.370,0.04768,NaN,NaN,NaN
175,2015-02-25,510050C1509M02400,C,0.575342,2.40,0.1780,2.370,0.04768,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
40724,2018-09-19,510050C1812M02950,C,0.268493,2.95,0.0121,2.538,0.02668,NaN,NaN,NaN
40727,2018-09-19,510050C1812M02900,C,0.268493,2.90,0.0165,2.538,0.02668,NaN,NaN,NaN
40743,2018-09-20,510050C1810M02750,C,0.093151,2.75,0.0050,2.541,0.02659,NaN,NaN,NaN
40762,2018-09-20,510050C1812M02950,C,0.265753,2.95,0.0110,2.541,0.02659,NaN,NaN,NaN


In [119]:
df_c

,trade_date,ts_code,call_put,t,k,c,s,r
0,2015-02-09,510050C1503M02200,C,0.120548,2.20,0.1826,2.331,0.04335
1,2015-02-09,510050C1506M02300,C,0.369863,2.30,0.2105,2.331,0.04335
2,2015-02-09,510050C1506M02350,C,0.369863,2.35,0.1870,2.331,0.04335
3,2015-02-09,510050C1506M02400,C,0.369863,2.40,0.1652,2.331,0.04335
9,2015-02-09,510050C1509M02200,C,0.619178,2.20,0.3130,2.331,0.04335
...,...,...,...,...,...,...,...,...
83285,2018-09-21,510050C1812M02450,C,0.263014,2.45,0.2340,2.632,0.02654
83294,2018-09-21,510050C1812M02900,C,0.263014,2.90,0.0277,2.632,0.02654
83295,2018-09-21,510050C1812M02850,C,0.263014,2.85,0.0369,2.632,0.02654
83296,2018-09-21,510050C1812M02750,C,0.263014,2.75,0.0662,2.632,0.02654


In [118]:
df_p

,trade_date,ts_code_p,call_put_p,t,k,c_p,s,r
4,2015-02-09,510050P1506M02200,P,0.369863,2.20,0.1379,2.331,0.04335
5,2015-02-09,510050P1506M02250,P,0.369863,2.25,0.1631,2.331,0.04335
6,2015-02-09,510050P1506M02300,P,0.369863,2.30,0.1882,2.331,0.04335
7,2015-02-09,510050P1506M02350,P,0.369863,2.35,0.2055,2.331,0.04335
8,2015-02-09,510050P1506M02400,P,0.369863,2.40,0.2329,2.331,0.04335
...,...,...,...,...,...,...,...,...
83290,2018-09-21,510050P1812M02650,P,0.263014,2.65,0.1143,2.632,0.02654
83291,2018-09-21,510050P1812M02600,P,0.263014,2.60,0.0890,2.632,0.02654
83292,2018-09-21,510050P1812M02550,P,0.263014,2.55,0.0709,2.632,0.02654
83293,2018-09-21,510050P1812M02500,P,0.263014,2.50,0.0538,2.632,0.02654


In [107]:
df_c = data_clear(df)

In [116]:
df_c[(df_c.ts_code == '510050P1612A02450') & (df_c.trade_date == '2016-11-29')]

,trade_date,ts_code,call_put,t,k,c,s,r,q
14860,2016-11-29,510050P1612A02450,P,0.079452,2.45,0.0248,2.446,0.02481,-0.057885
14926,2016-11-29,510050P1612A02450,P,0.079452,2.45,0.0248,2.446,0.02481,-0.204035


In [115]:
df_c[df_c.isnull().T.any()]

,trade_date,ts_code,call_put,t,k,c,s,r,q
128,2015-02-16,510050C1506M02500,C,0.350685,2.50,0.0979,2.394,0.04700,NaN
130,2015-02-16,510050C1506M02450,C,0.350685,2.45,0.1166,2.394,0.04700,NaN
132,2015-02-16,510050C1506M02400,C,0.350685,2.40,0.1390,2.394,0.04700,NaN
174,2015-02-25,510050C1509M02350,C,0.575342,2.35,0.2001,2.370,0.04768,NaN
175,2015-02-25,510050C1509M02400,C,0.575342,2.40,0.1780,2.370,0.04768,NaN
...,...,...,...,...,...,...,...,...,...
40724,2018-09-19,510050C1812M02950,C,0.268493,2.95,0.0121,2.538,0.02668,NaN
40727,2018-09-19,510050C1812M02900,C,0.268493,2.90,0.0165,2.538,0.02668,NaN
40743,2018-09-20,510050C1810M02750,C,0.093151,2.75,0.0050,2.541,0.02659,NaN
40762,2018-09-20,510050C1812M02950,C,0.265753,2.95,0.0110,2.541,0.02659,NaN


In [114]:
df_c = df_c.dropna(subset=['trade_date','ts_code'])

In [103]:
c_df

,trade_date,ts_code,call_put,t,k,c,s,r,q
0,2015-02-09,510050P1503M02200,C,0.120548,2.20,0.1826,2.331,0.04335,0.077108
1,2015-02-09,510050P1506M02300,C,0.369863,2.30,0.2105,2.331,0.04335,0.053040
2,2015-02-09,510050P1506M02350,C,0.369863,2.35,0.1870,2.331,0.04335,0.043117
3,2015-02-09,510050P1506M02400,C,0.369863,2.40,0.1652,2.331,0.04335,0.043111
4,2015-02-09,510050P1509M02200,C,0.619178,2.20,0.3130,2.331,0.04335,0.039107
...,...,...,...,...,...,...,...,...,...
36110,2018-09-21,510050P1812M02450,P,0.263014,2.45,0.0411,2.632,0.02654,0.008883
36111,2018-09-21,510050P1812M02900,P,0.263014,2.90,0.2815,2.632,0.02654,0.008638
36112,2018-09-21,510050P1812M02850,P,0.263014,2.85,0.2411,2.632,0.02654,0.008713
36113,2018-09-21,510050P1812M02750,P,0.263014,2.75,0.1717,2.632,0.02654,0.009588


In [104]:
c_df[(c_df.ts_code == '510050P1612A02450') & (c_df.trade_date == '2016-11-29')]

,trade_date,ts_code,call_put,t,k,c,s,r,q
14900,2016-11-29,510050P1612A02450,C,0.079452,2.45,0.0656,2.446,0.02481,-0.204035
14900,2016-11-29,510050P1612A02450,P,0.079452,2.45,0.0248,2.446,0.02481,-0.204035


In [77]:
df_c = df[df['call_put']=='C']
df_p = df[df['call_put']=='P']

In [78]:
def P_C(df):
    df.ts_code = df.ts_code[:6] + 'P' + df.ts_code[-10:]
    return df

In [79]:
df_c = df_c.apply(P_C,axis = 1)

In [80]:
df_p = df_p.rename(columns={'c':'c_p',
                     'call_put':'cal l_put_p'})
df = pd.merge(df_c,df_p,how='left',on=['trade_date','ts_code','k','t','r','s'])

In [85]:
df

,trade_date,ts_code,call_put,t,k,c,s,r,cal l_put_p,c_p
0,2015-02-09,510050P1503M02200,C,0.120548,2.20,0.1826,2.331,0.04335,P,0.0617
1,2015-02-09,510050P1506M02300,C,0.369863,2.30,0.2105,2.331,0.04335,P,0.1882
2,2015-02-09,510050P1506M02350,C,0.369863,2.35,0.1870,2.331,0.04335,P,0.2055
3,2015-02-09,510050P1506M02400,C,0.369863,2.40,0.1652,2.331,0.04335,P,0.2329
4,2015-02-09,510050P1509M02200,C,0.619178,2.20,0.3130,2.331,0.04335,P,0.1795
...,...,...,...,...,...,...,...,...,...,...
36110,2018-09-21,510050P1812M02450,C,0.263014,2.45,0.2340,2.632,0.02654,P,0.0411
36111,2018-09-21,510050P1812M02900,C,0.263014,2.90,0.0277,2.632,0.02654,P,0.2815
36112,2018-09-21,510050P1812M02850,C,0.263014,2.85,0.0369,2.632,0.02654,P,0.2411
36113,2018-09-21,510050P1812M02750,C,0.263014,2.75,0.0662,2.632,0.02654,P,0.1717


In [86]:
data_clear(df)

AttributeError: 'Series' object has no attribute 'c_p'

In [38]:
df[(df.ts_code == '510050P1612A02450') & (df.trade_date == '2016-11-29')]

,trade_date,ts_code,call_put,t,k,c,s,r
36461,2016-11-29,510050P1612A02450,P,0.079452,2.45,0.0248,2.446,0.02481


In [39]:
df[(df.ts_code == '510050P1612M02450') & (df.trade_date == '2016-11-29')]

,trade_date,ts_code,call_put,t,k,c,s,r
36366,2016-11-29,510050P1612M02450,P,0.079452,2.45,0.0456,2.446,0.02481


In [31]:
def iq(df): # 计算隐含分红率
    #q = -log((df.settle+df.exercise_price*exp(-df.interest*df.delta)-df.settle_p)/(df.s0))/df.delta
    q = -log((df.c+df.k*exp(-df.r*df.t)-df.c_p)/(df.s))/df.t
    return q


In [32]:
df_c = df[df['call_put']=='C']
df_p = df[df['call_put']=='P']

df_p = df_p.rename(columns={'c':'c_p','ts_code':'ts_code_p',
                     'call_put':'cal l_put_p'})
df = pd.merge(df_c,df_p,how='left',on=['trade_date','k','t','r','s'])

In [49]:
c_df[(c_df.trade_date=='2016-11-29') & (c_df.ts_code=='510050C1612M02450')]

,trade_date,ts_code,call_put,t,k,c,s,r,q
14859,2016-11-29,510050C1612M02450,C,0.079452,2.45,0.0369,2.446,0.02481,0.049106
14860,2016-11-29,510050C1612M02450,C,0.079452,2.45,0.0369,2.446,0.02481,-0.057885


In [50]:
c_df[(c_df.trade_date=='2016-11-29') & (c_df.ts_code=='510050P1612M02450')]

,trade_date,ts_code,call_put,t,k,c,s,r,q
14859,2016-11-29,510050P1612M02450,P,0.079452,2.45,0.0456,2.446,0.02481,0.049106
14925,2016-11-29,510050P1612M02450,P,0.079452,2.45,0.0456,2.446,0.02481,-0.098284


In [51]:
0.0369 -0.098284 

-0.061383999999999994

In [41]:
df[(df.trade_date=='2016-11-29') & (df.ts_code=='510050C1612A02450')]

,trade_date,ts_code,call_put,t,k,c,s,r
36446,2016-11-29,510050C1612A02450,C,0.079452,2.45,0.0656,2.446,0.02481


In [219]:
df[(df.trade_date=='2016-11-29') & (df.ts_code=='510050C1612M02450')]

,trade_date,ts_code,call_put,t,k,c,s,r,ts_code_p,cal l_put_p,c_p
14859,2016-11-29,510050C1612M02450,C,0.079452,2.45,0.0369,2.446,0.02481,510050P1612M02450,P,0.0456
14860,2016-11-29,510050C1612M02450,C,0.079452,2.45,0.0369,2.446,0.02481,510050P1612A02450,P,0.0248


In [74]:
df_c.drop_duplicates(subset=['trade_date','ts_code'])

,trade_date,ts_code,call_put,t,k,c,s,r,q
0,2015-02-09,510050C1503M02200,C,0.120548,2.20,0.1826,2.331,0.04335,0.077108
1,2015-02-09,510050C1506M02300,C,0.369863,2.30,0.2105,2.331,0.04335,0.053040
2,2015-02-09,510050C1506M02350,C,0.369863,2.35,0.1870,2.331,0.04335,0.043117
3,2015-02-09,510050C1506M02400,C,0.369863,2.40,0.1652,2.331,0.04335,0.043111
4,2015-02-09,510050C1509M02200,C,0.619178,2.20,0.3130,2.331,0.04335,0.039107
...,...,...,...,...,...,...,...,...,...
40799,2018-09-21,510050P1812M02450,P,0.263014,2.45,0.0411,2.632,0.02654,0.008883
40800,2018-09-21,510050P1812M02900,P,0.263014,2.90,0.2815,2.632,0.02654,0.008638
40801,2018-09-21,510050P1812M02850,P,0.263014,2.85,0.2411,2.632,0.02654,0.008713
40802,2018-09-21,510050P1812M02750,P,0.263014,2.75,0.1717,2.632,0.02654,0.009588


In [69]:
df.dropna(axis=1,how='any')

,trade_date,ts_code,call_put,t,k,c,s,r
0,2015-02-09,510050C1503M02200,C,0.120548,2.20,0.1826,2.331,0.04335
1,2015-02-09,510050C1506M02300,C,0.369863,2.30,0.2105,2.331,0.04335
2,2015-02-09,510050C1506M02350,C,0.369863,2.35,0.1870,2.331,0.04335
3,2015-02-09,510050C1506M02400,C,0.369863,2.40,0.1652,2.331,0.04335
4,2015-02-09,510050P1506M02200,P,0.369863,2.20,0.1379,2.331,0.04335
...,...,...,...,...,...,...,...,...
83294,2018-09-21,510050C1812M02900,C,0.263014,2.90,0.0277,2.632,0.02654
83295,2018-09-21,510050C1812M02850,C,0.263014,2.85,0.0369,2.632,0.02654
83296,2018-09-21,510050C1812M02750,C,0.263014,2.75,0.0662,2.632,0.02654
83297,2018-09-21,510050P1812M02900,P,0.263014,2.90,0.2815,2.632,0.02654


In [55]:
df_c = df.duplicated()dd

In [65]:
df_c = df_c.dropna(axis=0,how='any')
df_c

0        False
1        False
2        False
3        False
4        False
         ...  
83294    False
83295    False
83296    False
83297    False
83298    False
Length: 70232, dtype: bool

计算iv

计算iv

In [ ]:
long_time = df_c.groupby('trade_date').count().index.astype('str').tolist()

,trade_date,ts_code,call_put,t,k,c,s,r,q


In [ ]:
df_final = df_c.copy().reset_index(drop=True)

In [ ]:
d = pd.DataFrame()

In [ ]:
# 测试外连接

day = long_time[-1] # '2018-09-21'
# df_final = df_c.copy()
d_test = df_c[df_c.trade_date==day]
d_test = cal_iv(d_test)
d_test['ts_code']=d_test.index
d_test = d_test.reset_index(drop=True)
#print(d_test.shape)
#d = d.append(d_test)
## df_final = pd.merge(df_final,d_test,on=df_final.columns.to_list(),how='outer')

In [41]:
#根据BS公式计算期权价值
def bsm_value(s,k,t,r,sigma,q,option_type):
    d1 = ( log( s/k ) + ( r -q + 0.5*sigma**2 )*t )/( sigma*sqrt(t) )
    d2 = ( log( s/k ) + ( r -q - 0.5*sigma**2 )*t )/( sigma*sqrt(t) )
    if option_type.lower() == 'c':
        value = (s*exp(-q*t)*stats.norm.cdf( d1) - k*exp( -r*t )*stats.norm.cdf( d2))
    else:
        value = k * exp(-r * t) * stats.norm.cdf(-d2) - s*exp(-q*t) * stats.norm.cdf(-d1)
    return value

#二分法求隐含波动率
def bsm_imp_vol(s,k,t,r,c,q,option_type):
    # print(c)
    c_est = 0 # 期权价格估计值
    top = 1  #波动率上限
    floor = 0  #波动率下限
    sigma = ( floor + top )/2 #波动率初始值
    count = 0 # 计数器
    while abs( c - c_est ) > 0.000001:
        c_est = bsm_value(s,k,t,r,sigma,q,option_type) 
        #根据价格判断波动率是被低估还是高估，并对波动率做修正
        count += 1           
        if count > 100: # 时间价值为0的期权是算不出隐含波动率的，因此迭代到一定次数就不再迭代了
            sigma = 0
            break
        
        if c - c_est > 0: #f(x)>0
            floor = sigma
            sigma = ( sigma + top )/2
        else:
            top = sigma
            sigma = ( sigma + floor )/2
    return sigma  

def cal_iv(df): # 计算主程序
    option_list = df.ts_code.tolist()
    # print(option_list)
    df = df.set_index('ts_code')
    alist = []

    for option in option_list:
        s = df.loc[option,'s']
        k = df.loc[option,'k']
        t = df.loc[option,'t']
        r = df.loc[option,'r']
        c = df.loc[option,'c']
        q = df.loc[option,'q']
        option_type = df.loc[option,'call_put']
        sigma = bsm_imp_vol(s,k,t,r,c,q,option_type)
        alist.append(sigma)
    df['iv'] = alist
    return df  

In [43]:
def cal_v(df,long_time):
    df_final = pd.DataFrame()
    for ti in long_time:
        d_temp = df[df.trade_date==ti]
        d_temp = cal_iv(d_temp) # 计算隐含波动率
        # print(d_temp)
        # 处理日期
        d_temp['ts_code']=d_temp.index
        d_temp = d_temp.reset_index(drop=True)
        df_final = df_final.append(d_temp)
        df_final = df_final.reset_index(drop=True)
    return df_final

In [44]:
df_final_iv = cal_v(df_final,long_time)

In [45]:
df_final_iv

,trade_date,call_put,t,k,c,s,r,q,iv,ts_code
0,2015-02-09,C,0.120548,2.20,0.1826,2.331,0.04335,0.077108,0.359585,510050C1503M02200
1,2015-02-09,C,0.369863,2.30,0.2105,2.331,0.04335,0.053040,0.361618,510050C1506M02300
2,2015-02-09,C,0.369863,2.35,0.1870,2.331,0.04335,0.043117,0.351591,510050C1506M02350
3,2015-02-09,C,0.369863,2.40,0.1652,2.331,0.04335,0.043111,0.349829,510050C1506M02400
4,2015-02-09,C,0.619178,2.20,0.3130,2.331,0.04335,0.039107,0.347905,510050C1509M02200
...,...,...,...,...,...,...,...,...,...,...
58908,2018-09-21,P,0.263014,2.45,0.0411,2.632,0.02654,0.008883,0.216892,510050P1812M02450
58909,2018-09-21,P,0.263014,2.90,0.2815,2.632,0.02654,0.008638,0.198334,510050P1812M02900
58910,2018-09-21,P,0.263014,2.85,0.2411,2.632,0.02654,0.008713,0.197594,510050P1812M02850
58911,2018-09-21,P,0.263014,2.75,0.1717,2.632,0.02654,0.009588,0.202822,510050P1812M02750


# 把成交量也写上

In [46]:
df3 = df2.rename(columns={'date':'trade_date','trade_code':'ts_code','成交量':'volume'})
df3 = df3[['trade_date','ts_code','volume']].reset_index(drop=True)

In [47]:
df_pre = pd.merge(df_final_iv,df3,on=['trade_date','ts_code'],how='left')

In [48]:
df_pre

,trade_date,call_put,t,k,c,s,r,q,iv,ts_code,volume
0,2015-02-09,C,0.120548,2.20,0.1826,2.331,0.04335,0.077108,0.359585,510050C1503M02200,2501
1,2015-02-09,C,0.369863,2.30,0.2105,2.331,0.04335,0.053040,0.361618,510050C1506M02300,161
2,2015-02-09,C,0.369863,2.35,0.1870,2.331,0.04335,0.043117,0.351591,510050C1506M02350,106
3,2015-02-09,C,0.369863,2.40,0.1652,2.331,0.04335,0.043111,0.349829,510050C1506M02400,225
4,2015-02-09,C,0.619178,2.20,0.3130,2.331,0.04335,0.039107,0.347905,510050C1509M02200,247
...,...,...,...,...,...,...,...,...,...,...,...
58908,2018-09-21,P,0.263014,2.45,0.0411,2.632,0.02654,0.008883,0.216892,510050P1812M02450,3576
58909,2018-09-21,P,0.263014,2.90,0.2815,2.632,0.02654,0.008638,0.198334,510050P1812M02900,2337
58910,2018-09-21,P,0.263014,2.85,0.2411,2.632,0.02654,0.008713,0.197594,510050P1812M02850,529
58911,2018-09-21,P,0.263014,2.75,0.1717,2.632,0.02654,0.009588,0.202822,510050P1812M02750,1694


In [49]:
# 重新排一下序
t_columns = ['trade_date', 'ts_code', 'call_put','volume','t', 'k', 'c', 's', 'r', 'q','iv']
# df_final.reindex(columns=t_column)
df_pre = df_pre.reindex(columns=t_columns)

In [50]:
df_pre

,trade_date,ts_code,call_put,volume,t,k,c,s,r,q,iv
0,2015-02-09,510050C1503M02200,C,2501,0.120548,2.20,0.1826,2.331,0.04335,0.077108,0.359585
1,2015-02-09,510050C1506M02300,C,161,0.369863,2.30,0.2105,2.331,0.04335,0.053040,0.361618
2,2015-02-09,510050C1506M02350,C,106,0.369863,2.35,0.1870,2.331,0.04335,0.043117,0.351591
3,2015-02-09,510050C1506M02400,C,225,0.369863,2.40,0.1652,2.331,0.04335,0.043111,0.349829
4,2015-02-09,510050C1509M02200,C,247,0.619178,2.20,0.3130,2.331,0.04335,0.039107,0.347905
...,...,...,...,...,...,...,...,...,...,...,...
58908,2018-09-21,510050P1812M02450,P,3576,0.263014,2.45,0.0411,2.632,0.02654,0.008883,0.216892
58909,2018-09-21,510050P1812M02900,P,2337,0.263014,2.90,0.2815,2.632,0.02654,0.008638,0.198334
58910,2018-09-21,510050P1812M02850,P,529,0.263014,2.85,0.2411,2.632,0.02654,0.008713,0.197594
58911,2018-09-21,510050P1812M02750,P,1694,0.263014,2.75,0.1717,2.632,0.02654,0.009588,0.202822


# Mi 在值程度

In [51]:
def m_cal(df):
    m = log(df.k/df.s*exp(df.q*df.t))
    return m 

In [52]:
df_m = df_pre.copy()
df_m['M'] = df_m.apply(m_cal, axis=1) # 计算在值程度
c_put = df_m[df_m['call_put']=='C']
p_cal = df_m[df_m['call_put']=='P']

In [53]:
df_m

,trade_date,ts_code,call_put,volume,t,k,c,s,r,q,iv,M
0,2015-02-09,510050C1503M02200,C,2501,0.120548,2.20,0.1826,2.331,0.04335,0.077108,0.359585,-0.048545
1,2015-02-09,510050C1506M02300,C,161,0.369863,2.30,0.2105,2.331,0.04335,0.053040,0.361618,0.006229
2,2015-02-09,510050C1506M02350,C,106,0.369863,2.35,0.1870,2.331,0.04335,0.043117,0.351591,0.024065
3,2015-02-09,510050C1506M02400,C,225,0.369863,2.40,0.1652,2.331,0.04335,0.043111,0.349829,0.045117
4,2015-02-09,510050C1509M02200,C,247,0.619178,2.20,0.3130,2.331,0.04335,0.039107,0.347905,-0.033626
...,...,...,...,...,...,...,...,...,...,...,...,...
58908,2018-09-21,510050P1812M02450,P,3576,0.263014,2.45,0.0411,2.632,0.02654,0.008883,0.216892,-0.069320
58909,2018-09-21,510050P1812M02900,P,2337,0.263014,2.90,0.2815,2.632,0.02654,0.008638,0.198334,0.099239
58910,2018-09-21,510050P1812M02850,P,529,0.263014,2.85,0.2411,2.632,0.02654,0.008713,0.197594,0.081867
58911,2018-09-21,510050P1812M02750,P,1694,0.263014,2.75,0.1717,2.632,0.02654,0.009588,0.202822,0.046379


In [58]:
df_m.to_csv('iv_data.csv')

In [54]:
p_call = p_cal.copy()
p_call.loc[:,'M'] = p_cal.M.apply(lambda x: -x)

In [55]:
def sample_count(df):
    return [df.shape[0],round(df.iv.mean(),4)]

In [56]:
# 成交量百分比
def volume_percent(df):
    deep_s = df_m.M<-0.12
    normal_s = (df_m.M > -0.12) & (df_m.M <= -0.03)
    equal_s = (df_m.M > -0.03) & (df_m.M < 0.03)
    virtual_s = (df_m.M >= 0.03) & (df_m.M < 0.12)
    high_s = df_m.M > 0.12
    return [round(df[i].volume.sum() / df_m.volume.sum(),4)
            for i in [deep_s,normal_s,equal_s,virtual_s,high_s]]

In [57]:
volume_p = volume_percent(df_m)

In [733]:
def sample_percent(df,type_s):
    return round(df[type_s].volume.sum() / c_put.volume.sum() * 100,2) 

In [734]:
sample_percent(c_put,deep_s)

0.36

In [432]:
sample_count(c_put[deep_s & short])

[576, 0.3541]

In [832]:
# 样本统计 
def sample_count(df):
    return [df.shape[0],round(df.iv.mean(),4)]

def descriptive(df,type_s):
    if type_s == 'C':
        temp = df[df['call_put']=='C']
        # print(temp.shape)
        # print(df.shape)
    else:
        temp = df[df['call_put']=='P'].copy()
        temp.loc[:,'M'] = df.M.apply(lambda x: -x) # 看跌期权在值与看涨相反
        # print(temp.shape)
    # def sample_percent(df,type_s):
    #     return round(df[type_s].volume.sum() / c_put.volume.sum() * 100,2)
    # 长短期划分
    short = ((temp.t*365) < 60)
    medium = (temp.t*365 >= 60) & (temp.t*365 <= 180)
    long = ((temp.t*365) > 180)
    
    # 设定上下限
    top = 0.12 
    mid = 0.03
    
    # 深度实值
    deep_s = temp.M<=-top
    s_1 = temp[deep_s & short]
    m_1 = temp[deep_s & medium]
    l_1 = temp[deep_s & long]
    deep_1ist = [sample_count(i) for i in [s_1,m_1,l_1]]

        # normal_s = (c_put.M > -0.12) & (c_put.M <= -0.03)
    # 实值
    # print(temp.shape)
    normal_s = ((temp.M>-top)&(temp.M<=-mid))

    s_2 = temp[normal_s & short]
    # print(s_2.shape)
    m_2 = temp[normal_s & medium]
    l_2 = temp[normal_s & long]
    normal_1ist = [sample_count(i) for i in [s_2,m_2,l_2]]
    # print(s_2)
    
    # 平值
    equal_s = (temp.M>-mid)&(temp.M<mid)
    s_3 = temp[equal_s & short]
    m_3 = temp[equal_s & medium]
    l_3 = temp[equal_s & long]
    equal_1ist = [sample_count(i) for i in [s_3,m_3,l_3]]
    
    # 虚值
    virtual_s = (temp.M>=mid)&(temp.M<top)
    s_4 = temp[virtual_s & short]
    m_4 = temp[virtual_s & medium]
    l_4 = temp[virtual_s & long]
    virtual_1ist = [sample_count(i) for i in [s_4,m_4,l_4]]
    
    
    #深度虚值
    high_s = temp.M>=top
    s_5 = temp[high_s & short]
    m_5 = temp[high_s & medium]
    l_5 = temp[high_s & long]
    high_1ist = [sample_count(i) for i in [s_5,m_5,l_5]]
    
    num = [deep_1ist,normal_1ist,equal_1ist, virtual_1ist, high_1ist]
    return num

In [833]:
des_call = descriptive(df_m,'C')
des_put = descriptive(df_m,'P')

描述性统计 des_call 看涨期权， des_put 看跌期权

In [835]:
des_put,des_call

([[[1303, 0.4493], [1097, 0.3454], [184, 0.2995]],
  [[3245, 0.2774], [3050, 0.2276], [1432, 0.2078]],
  [[4357, 0.2208], [3464, 0.1972], [1566, 0.1902]],
  [[3377, 0.254], [3270, 0.1982], [1078, 0.1932]],
  [[578, 0.3526], [946, 0.2428], [38, 0.2693]]],
 [[[576, 0.3541], [945, 0.243], [38, 0.2693]],
  [[3316, 0.2553], [3191, 0.2021], [1080, 0.1953]],
  [[4573, 0.2195], [3445, 0.2005], [1552, 0.1933]],
  [[3617, 0.2697], [3499, 0.2231], [1452, 0.2089]],
  [[1313, 0.4483], [1146, 0.3424], [185, 0.2992]]])

In [836]:
sample_volume = df_m.shape[0]
trade_volume = df_m.volume.sum()
print(sample_volume,trade_volume)

58913 362035678


In [849]:
sample_count= []
for i in zip(des_call,des_put):
    sample_count.append(i[0][0][0]+i[0][1][0]+i[0][2][0]+i[1][0][0]+i[1][1][0]+i[1][2][0])
sample_percent = [round(i / sample_volume,4) for i in sample_count]

In [848]:
sample_count

[4143, 15314, 18957, 16293, 4206]

In [850]:
volume_p,sample_percent

([0.0076, 0.1696, 0.62, 0.1885, 0.0143],
 [0.0703, 0.2599, 0.3218, 0.2766, 0.0714])

In [816]:
sample_percent

[0.0703240371395108,
 0.2599426272639316,
 0.32177957326905776,
 0.2765603517050566,
 0.07139341062244327]

In [633]:
df_shibor.date.astype('str').tolist()

['2015-01-04',
 '2015-01-05',
 '2015-01-06',
 '2015-01-07',
 '2015-01-08',
 '2015-01-09',
 '2015-01-12',
 '2015-01-13',
 '2015-01-14',
 '2015-01-15',
 '2015-01-16',
 '2015-01-19',
 '2015-01-20',
 '2015-01-21',
 '2015-01-22',
 '2015-01-23',
 '2015-01-26',
 '2015-01-27',
 '2015-01-28',
 '2015-01-29',
 '2015-01-30',
 '2015-02-02',
 '2015-02-03',
 '2015-02-04',
 '2015-02-05',
 '2015-02-06',
 '2015-02-09',
 '2015-02-10',
 '2015-02-11',
 '2015-02-12',
 '2015-02-13',
 '2015-02-15',
 '2015-02-16',
 '2015-02-17',
 '2015-02-25',
 '2015-02-26',
 '2015-02-27',
 '2015-02-28',
 '2015-03-02',
 '2015-03-03',
 '2015-03-04',
 '2015-03-05',
 '2015-03-06',
 '2015-03-09',
 '2015-03-10',
 '2015-03-11',
 '2015-03-12',
 '2015-03-13',
 '2015-03-16',
 '2015-03-17',
 '2015-03-18',
 '2015-03-19',
 '2015-03-20',
 '2015-03-23',
 '2015-03-24',
 '2015-03-25',
 '2015-03-26',
 '2015-03-27',
 '2015-03-30',
 '2015-03-31',
 '2015-04-01',
 '2015-04-02',
 '2015-04-03',
 '2015-04-07',
 '2015-04-08',
 '2015-04-09',
 '2015-04-

In [ ]:
for day in df_shibor.date.astype('str').tolist():
    if df_c[df_c.trade_date == day].shape[0]>50:
        print(day)

In [60]:
df_t = df_c.copy()

# 画图

In [104]:
date_info = '2018-07-06'
df = cal_iv(df_t[df_t.trade_date == date_info]) # 计算隐含波动率
# pd.merge(df_t,df,how='left',on=['trade_date'])
df['ts_code']=df.index
df = df.reset_index(drop=True)
df = data_pivot(df) # 数据透视表
df = fitting(df)
#smile_plot(df)
df

t,k,19,47,82,173
0,2.20,0.339531,0.288300,0.265579,0.251326
1,2.25,0.317879,0.278305,0.258583,0.250393
2,2.30,0.303871,0.266098,0.255497,0.248653
3,2.35,0.291206,0.263565,0.251984,0.246998
4,2.40,0.274021,0.254444,0.248363,0.247948
5,2.45,0.264668,0.251553,0.247719,0.246254
6,2.50,0.255569,0.249901,0.246162,0.242655
7,2.55,0.254272,0.247253,0.247318,0.240807
8,2.60,0.258240,0.246952,0.252037,0.243767
9,2.65,0.263412,0.249523,0.251305,0.240675


In [70]:
df = cal_iv(df_t[df_t.trade_date == date_info]) 
df

,trade_date,call_put,t,k,c,s,r,q,iv
ts_code,,,,,,,,,
510050C1809M02350,2018-07-06,C,0.224658,2.35,0.1553,2.425,0.02588,0.022678,0.251984
510050C1812M02350,2018-07-06,C,0.473973,2.35,0.2042,2.425,0.02588,0.019184,0.246998
510050C1807M02300,2018-07-06,C,0.052055,2.30,0.1445,2.425,0.02588,0.038828,0.303871
510050C1809M02300,2018-07-06,C,0.224658,2.30,0.1887,2.425,0.02588,0.019193,0.255497
510050C1812M02300,2018-07-06,C,0.473973,2.30,0.2322,2.425,0.02588,0.020845,0.248653
...,...,...,...,...,...,...,...,...,...
510050P1812M02600,2018-07-06,P,0.473973,2.60,0.2616,2.425,0.02588,0.019051,0.243767
510050P1812M02800,2018-07-06,P,0.473973,2.80,0.4009,2.425,0.02588,0.010323,0.238531
510050P1812M02850,2018-07-06,P,0.473973,2.85,0.4478,2.425,0.02588,0.014795,0.242117


In [869]:
df

t,k,19,47,82,173
0,2.20,0.339531,0.288300,0.265579,0.251326
1,2.25,0.317879,0.278305,0.258583,0.250393
2,2.30,0.303871,0.266098,0.255497,0.248653
3,2.35,0.291206,0.263565,0.251984,0.246998
4,2.40,0.274021,0.254444,0.248363,0.247948
5,2.45,0.264668,0.251553,0.247719,0.246254
6,2.50,0.255569,0.249901,0.246162,0.242655
7,2.55,0.254272,0.247253,0.247318,0.240807
8,2.60,0.258240,0.246952,0.252037,0.243767
9,2.65,0.263412,0.249523,0.251305,0.240675


In [ ]:
df_t1 = pd.merge(df_t,df,on=df_t.columns.to_list(),how='left')

In [ ]:
df_t1.dropna()

In [62]:
def data_pivot(df): # 数据透视
    df = df.reset_index()
    option_type = 'C' # 具有相同执行价格、相同剩余到期时间的看涨看跌期权隐含波动率相等，因此算一个就够了
    df = df[df['call_put']==option_type]
    df = df.drop(['ts_code','trade_date','c','s','r','call_put','q'],axis=1)
    df['t'] = df['t']*365
    df['t'] = df['t'].astype(int)
    df = df.pivot_table(index=["k"],columns=["t"],values=["iv"])
    df.columns = df.columns.droplevel(0)
    df.index.name = None
    df = df.reset_index()
    df = df.rename(columns={'index':'k'})

    return df

def fitting(df): # 多项式拟合    
    col_list = df.columns
    for i in range(df.shape[1]-1):
        x_col = col_list[0]
        y_col = col_list[i+1]
        df1 = df.dropna(subset=[y_col])
        
        x = df1.iloc[:,0]
        y = df1.iloc[:,i+1]

        degree = 2
                
        weights = np.polyfit(x, y, degree)
        model = np.poly1d(weights)
        predict = np.poly1d(model)
        x_given_list = df[pd.isnull(df[y_col]) == True][x_col].tolist()
        # 所有空值对应的k组成列表
        for x_given in x_given_list:
            y_predict = predict(x_given)
            df.loc[df[x_col]==x_given, y_col] = y_predict
    return df

In [67]:
def im_surface(df): # 波动率曲面作图
    # df = plot_df()
    df = fitting(df)    
    #df.to_excel('iv_fitting.xlsx')
    df = df.set_index('k')

    y = np.array(df.index)
    x = np.array(df.columns)
    fig = go.Figure(data=[go.Surface(z=df.values, x=x, y=y)])

    fig.update_layout(scene = dict(
                    xaxis_title='剩余期限',
                    yaxis_title='执行价格',
                    zaxis_title='隐含波动率'),
                    width=1400,
                    margin=dict(r=20, b=10, l=10, t=10),
                    )
    # fig.write_image("fig1.jpg")
    plotly.offline.plot(fig)

def smile_plot(df): # 波动率微笑作图
    # df = plot_df()
    df = df.set_index('k')
    df = df.stack().reset_index()
    df.columns = ['k', 'days', 'iv']
    fig = px.line(df, x="k", y="iv", color="days",line_shape="spline")
    plotly.offline.plot(fig)

In [ ]:
def main(df_c,date):
   # df = extra_data(date) # 提取数据
   # df = data_clear(df) # 数据清洗4
    i_v = df_c[df_c.trade_date == date]
    df = cal_iv(i_v) # 计算隐含波动率
    df['ts_code']=df.index
    df = df.reset_index(drop=True)
    df_t = pd.merge(df_c,df,on=df_t.columns.to_list(),how='left')
    return df_t
    # df = data_pivot(df) # 数据透视表
    # df = fitting(df)
    # return df.iloc[:,-2]
    # smile_plot(df) # 波动率微笑
    # im_surface(df) # 波动率曲面
    
if __name__ == '__main__':
    # df_shibor.date.astype('str').tolist() # 时间序列
    long_time = df_shibor.date.astype('str').tolist()
    count = 0
    s_list = []
    for ti in long_time:
        date = ti 
        df_t = main(df_t,date)
        
   #     if d_[d_.values!=0].values.shape[0] != 0 :
   #         s_list.append(ti)
   #     else: 
   #         count += 1
   #         if count % 50 == 0:
   #             print('已筛选{}个，仍未发现'.format(count))
   # print(s_list)
   # # date = '2017-5-10'  # 可任意更改日期
    

In [ ]:
# df2.t = df2.t.values.round(4)
# df2.to_csv('test.csv')

指定日期输出 https://blog.csdn.net/weixin_45505706/article/details/106721590

In [ ]:
df_shibor.date.astype('str').tolist()